In [6]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API Key
#OPENAI_API_KEY = ""

In [9]:
# Use it in your LLM model
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("sec1.pdf")

from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=OPENAI_API_KEY)
 

# Split content into chunks.
# Import a splitter module called RecursiveCharacterTextSplitter:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Instantiate the splitter:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# Do the splitting:
splits = text_splitter.split_documents(loader.load())

# Embed and store splits in Chroma database.
# Import the Chroma and embeddings modules:
from langchain.vectorstores import Chroma
# from langchain.embeddings import OpenAIEmbeddings
# Instantiate the Chroma vector store with arguments:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model
)
# Instantiate a retriever based on the vector store:
retriever = vectorstore.as_retriever()

# Instantiate a prompt.
# We will use a standard RAG prompt downloaded from Langsmith:
# https://smith.langchain.com/hub/rlm/rag-prompt
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")


#llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# Instantiate RAG chain:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

completion = rag_chain.invoke("Who is the founder of Singapore?")
print(completion.content)


C:\Users\minli\AppData\Local\Temp\ipykernel_26780\1800588230.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
C:\Users\minli\anaconda3\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


NameError: name 'OpenAI' is not defined

In [12]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

C:\Users\minli\anaconda3\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [13]:
# Instantiate RAG chain:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

completion = rag_chain.invoke("Who is the founder of Singapore?")
print(completion.content)


The founder of Singapore is often recognized as Sir Stamford Raffles, who signed the 1819 Treaty allowing the British to establish a trading post in the southern part of Singapore. However, some argue that William Farquhar and John Crawfurd also played significant roles in the founding of Singapore through their respective treaties and contributions to the development of the island. Ultimately, the question of who founded Singapore may vary depending on different perspectives and interpretations of what it means to be a founder.


## trying with a custom prompt template

In [15]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

# Define prompt template
rag_prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are the Heritage Education Research Assistant, an AI-powered tool designed to help educators in Singapore create comprehensive and balanced lesson plans about Singapore's history and culture. Your task is to provide multiple perspectives on historical questions, with a focus on validated sources from the National Heritage Board (NHB) and other reputable institutions.

Given a user's question and any provided filters (student age group, historical timeframe, theme), please:

1. Generate 3-5 different perspectives on the question, each with a brief summary (2-3 sentences) explaining the reasoning behind that perspective.
2. Ensure that the language and content complexity is appropriate for the specified student age group (if provided).
3. If a specific historical timeframe or theme is specified, tailor your responses to fit within those parameters.
4. After presenting the perspectives, suggest 2-3 discussion questions that could encourage critical thinking among students about these different viewpoints.

Remember, your goal is to provide educators with balanced, well-sourced information that they can use to create engaging and thought-provoking lessons about Singapore's history and culture.

Context: {context}

Question: {question}
"""
)

# # Initialize LLM
# llm = ChatOpenAI()

# Ensure retriever only gets the question string, not a dictionary
def extract_question(inputs):
    return inputs["question"]

# Define RAG chain
rag_chain = (
    {"context": RunnableLambda(extract_question) | retriever, "question": RunnablePassthrough()}
    | rag_prompt_template
    | llm
)

# Invoke
completion = rag_chain.invoke({"question": "Who is the founder of Singapore?"})
print(completion.content)


Perspective 1: Sir Stamford Raffles
Summary: Some argue that Sir Stamford Raffles should be considered the founder of Singapore because he signed the 1819 Treaty that allowed the British to establish a trading post in the southern part of Singapore. Raffles' role in the establishment of Singapore is often highlighted in historical narratives.

Perspective 2: William Farquhar
Summary: Others contend that William Farquhar should be recognized as the founder of Singapore because he played a significant role in building Singapore from scratch. Farquhar's contributions to the early development of Singapore are noteworthy and cannot be overlooked.

Perspective 3: John Crawfurd
Summary: There are those who consider John Crawfurd as the founder of Singapore because he signed the 1824 Treaty of Friendship and Alliance that gave the British control over the entire island. Crawfurd's diplomatic efforts and contributions to the British control over Singapore are also significant.

Discussion Quest

In [17]:
completion = rag_chain.invoke({"question": "How has Singapore’s education system changed over the decades?"})
print(completion.content)

Perspective 1: Limited British Focus on Education (1819-1867)
Summary: During Singapore's early years under British rule, education was not a priority for the colonial government. The focus was primarily on trade and administration, leading to minimal investment in education and healthcare services for the local population.

Perspective 2: Growth of Chinese Schools and Missionary Schools (1867-1942)
Summary: In the late 19th and early 20th centuries, Singapore saw the emergence of Chinese schools established by the Chinese community and schools run by Christian missionaries. These institutions played a significant role in providing education to different segments of the population, with varying degrees of autonomy from government interference.

Perspective 3: Expansion of Educational Activities (1867-1942)
Summary: From 1867 onwards, there was a notable growth in educational activities in Singapore. This period marked an expansion of education based on the principle of meeting the evol

## Other Possible Sample Questions
Here are some alternative questions your RAG chatbot could answer, covering a range of historical, cultural, and societal topics related to Singapore:

Founding & Early History
How did Stamford Raffles’ role in Singapore’s founding compare to William Farquhar’s contributions?
What were the key reasons for the British choosing Singapore as a trading post in 1819?
What was Singapore like before British colonization?
How did indigenous communities like the Orang Laut contribute to early Singapore’s development?
Colonial & Wartime Singapore
How did British colonial rule impact Singapore’s economy and society?
What were the main causes and consequences of the 1915 Singapore Mutiny?
How did World War II and the Japanese occupation shape Singapore’s national identity?
What were the living conditions like for civilians during the Japanese Occupation?
Post-War & Independence
What were the key factors leading to Singapore’s independence from Malaysia in 1965?
How did Lee Kuan Yew’s leadership shape Singapore’s early years as an independent nation?
What were the major challenges Singapore faced immediately after independence?
How did Singapore’s economic policies in the 1960s and 70s contribute to its rapid development?
Immigration & Multiculturalism
How have different waves of immigration shaped Singapore’s cultural landscape?
What was the role of Chinese, Malay, Indian, and Eurasian communities in shaping Singapore’s identity?
How has Singapore’s housing policy influenced racial integration and social cohesion?
Social Policies & National Identity
What are the origins and significance of Singapore’s bilingual education policy?
How has the concept of ‘kiasu’ influenced Singaporean society?
What is the role of National Service in shaping Singaporean national identity?
How has the Speak Mandarin Campaign influenced Singapore’s linguistic landscape?
Economic & Urban Development
How did Singapore transition from a third-world country to a first-world nation?
What role has the Port of Singapore played in the country’s economic success?
How has urban planning shaped modern Singapore?
What were the key challenges and benefits of Singapore’s Smart Nation initiative?
Politics & Governance
How has the PAP maintained political dominance in Singapore since independence?
What were the key differences between the governance styles of Lee Kuan Yew, Goh Chok Tong, and Lee Hsien Loong?
How does Singapore’s approach to governance differ from other Southeast Asian nations?
What are the main checks and balances on political power in Singapore?
Environmental & Sustainability Issues
How has Singapore tackled water scarcity over the years?
What are some major environmental challenges Singapore faces today?
How has land reclamation shaped Singapore’s geography and economy?
What initiatives has Singapore undertaken to become a more sustainable city?
Culture, Arts & Heritage
How has Singapore’s hawker culture evolved over time?
What role do traditional festivals play in Singapore’s multicultural society?
How has the Peranakan culture influenced Singapore’s food, language, and traditions?
What is the significance of heritage conservation efforts in Singapore?
Education & Social Change
How has Singapore’s education system changed over the decades?
What are the pros and cons of the streaming system in Singaporean schools?
How does Singapore’s meritocratic system impact social mobility?
What role has technology played in transforming education in Singapore?
International Relations & Security
How has Singapore managed to maintain diplomatic neutrality in global conflicts?
What are the key aspects of Singapore’s defense strategy?
How does Singapore balance its relationships with the US, China, and ASEAN countries?
Would you like me to refine or categorize these questions differently based on your specific focus areas?